In [4]:
# mlp for multi-output regression
from numpy import mean
from numpy import std
from sklearn.datasets import make_regression
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense,Dropout, BatchNormalization
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.optimizers import SGD
from matplotlib import pyplot
from tensorflow.keras.metrics import RootMeanSquaredError
from keras import backend as K
from sklearn.model_selection import KFold
import numpy as np
from sklearn.multioutput import MultiOutputRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
import nltk
import pickle
from tensorflow import keras
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
# pip install plot_keras_history

In [6]:
# from plot_keras_history import plot_history

In [7]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [8]:
# pip install kora

In [9]:
from kora.drive import get_path
p = get_path('1Dlg2Ya4JarjrsTWW-dkA3BOkLjzNjnw6')

ModuleNotFoundError: ignored

In [ ]:
cd /content/gdrive/MyDrive/Colab Notebooks/Thesis Project 2021/'Regular Schedule NN model'

In [ ]:
df= pd.read_csv('data_concat.csv')
#df
X= df[df.columns[:8]].values
# y= df[['Electricity:Facility','Photovoltaic:ElectricityProduced','ElectricityPurchased:Facility']].values
y= df[['Electricity:Facility','Photovoltaic:ElectricityProduced']].values

In [ ]:
ax = df['ElectricityNet:Facility'].plot(figsize=(10, 5), style='.-', markerfacecolor='red')
ax.set_ylabel("Net Site Energy (kWh)")
ax.set_xlabel("Design Number")
ax.set_title("Annual Net Site Energy for Various Designs")

In [ ]:
# #y.mean(axis=0)
# y.min(axis=0)
# y.max(axis=0)

In [ ]:
# created scaler
scaler_x = StandardScaler()
# fit scaler on training dataset
scaler_x.fit(X)
# transform dataset
X_scaled = scaler_x.transform(X)

# created scaler
scaler_y = StandardScaler()
# fit scaler on training dataset
scaler_y.fit(y)
# transform dataset
y_scaled = scaler_y.transform(y)

In [ ]:
# split into train and test
n_train = 150
trainX, testX = X_scaled[:n_train, :], X_scaled[n_train:, :]
trainy, testy = y_scaled[:n_train], y_scaled[n_train:]

In [ ]:
def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [ ]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true - y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [ ]:
# Create the SVR regressor
svr = SVR(epsilon=0.1)

In [ ]:
# Create the Multioutput Regressor
mor = MultiOutputRegressor(svr)

In [ ]:
# Train the regressor
mor = mor.fit(trainX, trainy)

# Generate predictions for testing data
y_pred = mor.predict(testX)

# Evaluate the regressor
mse_one = mean_squared_error(testy[:,0], y_pred[:,0],squared=False)
mse_two = mean_squared_error(testy[:,1], y_pred[:,1],squared=False)
# mse_three = mean_squared_error(testy[:,2], y_pred[:,2],squared=False)
#print(f'RMSE for first regressor: {mse_one} - second regressor: {mse_two} - Third regressor: {mse_three}')
mae_one = mean_absolute_error(testy[:,0], y_pred[:,0])
mae_two = mean_absolute_error(testy[:,1], y_pred[:,1])
# mae_three = mean_absolute_error(testy[:,2], y_pred[:,2])
#print(f'MAE for first regressor: {mae_one} - second regressor: {mae_two} - third regressor: {mae_three}')

In [ ]:
# get the model
def get_model(n_inputs, n_outputs):
    model = Sequential()
    model.add(Dense(512, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
#     model.add(Dropout(0.5))
    model.add(Dense(256,activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(128,activation='relu'))
#     model.add(Dropout(0.2))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(32,activation='relu'))
    model.add(Dense(16,activation='relu'))
#     model.add(Dense(4,activation='relu'))
    #model.add(Dense(4,activation='relu'))
    model.add(Dense(n_outputs))
    optimizer = tf.keras.optimizers.Adam(0.0005)
    model.compile(loss=root_mean_squared_error, optimizer=optimizer, metrics=[r2_keras])
#     model.compile(loss=root_mean_squared_error, optimizer=SGD(lr=0.01, momentum=0.9))
    return model

In [ ]:

# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1

# Merge inputs and targets
inputs = np.concatenate((trainX, testX), axis=0)
targets = np.concatenate((trainy, testy), axis=0)


n_inputs, n_outputs = X.shape[1], y.shape[1]
# get model

for train, test in kfold.split(inputs, targets):
    model = get_model(n_inputs, n_outputs)
    # fit the model on all data
    history= model.fit(X_scaled[train], y_scaled[train],verbose=1, epochs=2500)
    # evaluate the model
    train_mse = model.evaluate(X_scaled[train], y_scaled[train], verbose=0)
    test_mse = model.evaluate(X_scaled[test], y_scaled[test], verbose=0)

    #print('Fold: %.1f, Train: %.3f, Test: %.3f' % (fold_no, train_mse, test_mse))
    # plot loss during training
    pyplot.title('Root Mean Squared Error')
    pyplot.plot(history.history['loss'], label='train')
    #pyplot.plot(history.history['val_loss'], label='test')
    pyplot.legend()
    pyplot.show()
    # Increase fold number
    fold_no = fold_no + 1
plot_history(history)

In [ ]:
pred= scaler_y.inverse_transform(model.predict(testX))
compY = scaler_y.inverse_transform(testy)

In [ ]:
# g=plt.scatter(y[:,1],predicted_y[:,1])
#'Electricity:Facility','Photovoltaic:ElectricityProduced','ElectricityPurchased:Facility'
plt.rcParams["figure.figsize"] = (15,7)
plt.title('Electricity Consumption: Facility')
plt.ylabel('ELectricity (kWh)')
plt.xlabel('Data Points')
plt.plot(compY[:,0], 'or',label='actual')
plt.plot(pred[:,0], '-', color='gray',label='predicted')
#plt.xlim(-1, 80)
plt.legend()
plt.show()

In [ ]:
def plot_prediction(test_labels, test_predictions):
  plt.figure()
  plt.scatter(test_labels, test_predictions, color='red')
  plt.xlabel('True Values [kWh]')
  plt.ylabel('Predictions [kWh]')
  plt.axis('equal')
  plt.xlim(plt.xlim())
  plt.ylim(plt.ylim())
  _ = plt.plot([-100, 100],[-100,100])

In [ ]:
plt.scatter(compY[:,0], pred[:,0], color='red')
plt.plot(np.unique(compY[:,0]), np.poly1d(np.polyfit(compY[:,0], pred[:,0], 1))(np.unique(compY[:,0])))
plt.xlabel('Actual Electical Consumption [kWh]')
plt.ylabel('Predicted Electrical Consumption [kWh]')

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams.update({'font.size': 12})
# matplotlib.rcParams.update({'font.family': 'Times New Roman'})
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.serif"] = "Times New Roman"

In [ ]:
plt.scatter(compY[:,0], pred[:,0], color='red', label ='actual vs predicted scatter points')
# plt.plot(np.unique(compY[:,0]), np.poly1d(np.polyfit(compY[:,0], pred[:,0], 1))(np.unique(compY[:,0])))
plt.plot(compY[:,0], compY[:,0], label = 'identity line')
plt.xlabel('Actual Electric Energy Consumption [kWh]')
plt.ylabel('Predicted Electric Energy Consumption [kWh]')
plt.legend()

# save model

In [ ]:
#saving weights and model
model.save("./my_model")

#save scaler
with open("my_scaler_x.pkl","wb") as f:
    pickle.dump(scaler_x, f)

with open("my_scaler_y.pkl","wb") as f:
    pickle.dump(scaler_y, f)

# test

In [ ]:
# It can be used to reconstruct the model identically.
reconstructed_model = keras.models.load_model("./my_model",compile=False)

#load scaler
with open("my_scaler_x.pkl","rb") as f:
    loaded_scaler_x= pickle.load(f)

with open("my_scaler_y.pkl","rb") as f:
    loaded_scaler_y= pickle.load(f)

In [ ]:
#let's check
pred= loaded_scaler_y.inverse_transform(reconstructed_model.predict(testX))